Define a custom NMF function:

In [1]:
def nmf(X, W, H, n_iter=100):
    import numpy as np
    for iter in range(n_iter):
        # update H
        H = H * np.divide(W.T @ X, W.T @ W @ H)
        # update W
        W = W * np.divide(X @ H.T, W @ H @ H.T)
    return W, H, {}

In [2]:
#
# Generate Data
#
import os
import sys; sys.path.append(os.path.join("..", "..", "scripts"))
from generate_X import gen_data

X = gen_data(R=4, shape=[10,20])["X"]
X.shape

/Users/maksim/Desktop/Code/telf_internal/examples/NMFk/../../scripts/generate_X.py:60: UserWarning: Kruskal's theorem probably won't apply, may not have a unique nCPD.
  warnings.warn("Kruskal's theorem probably won't apply, may not have a unique nCPD.")


(10, 20)

In [3]:
from TELF.factorization import NMFk
params = {
    "n_perturbs":10,
    "n_iters":10,
    "epsilon":0.015,
    "n_jobs":1,
    "init":"nnsvd", 
    "use_gpu":False,
    "save_path":os.path.join("..", "..", "results"),
    "save_output":True,
    "collect_output":True,
    "predict_k":True,
    "verbose":True,
    "transpose":False,
    "sill_thresh":0.9,
    "nmf_verbose":False,
    "nmf_method":"func",
    "nmf_func":nmf # custom function
}
model = NMFk(**params)

n_perturbs: 10
perturb_type: uniform
n_iters: 10
epsilon: 0.015
init: nnsvd
save_path: ../../results
save_output: True
use_gpu: False
verbose: True
nmf_verbose: False
perturb_verbose: False
transpose: False
collect_output: True
sill_thresh: 0.9
predict_k: True
predict_k_method: WH_sill
n_jobs: 1
n_nodes: 1
nmf: <function nmf at 0x10641f7e0>
nmf_method: func
nmf_obj_params: {}
clustering_obj_params: {}
pruned: True
calculate_error: True
consensus_mat: False
use_consensus_stopping: 0
mask: None
calculate_pac: False
simple_plot: True
get_plot_data: False
perturb_multiprocessing: False
k_search_method: linear
H_sill_thresh: None
factor_thresholding: None
factor_thresholding_H_regression: None
factor_thresholding_obj_params: {}
factor_thresholding_H_regression_obj_params: {}
clustering_method: kmeans
device: [-1]
lock: <unlocked _thread.lock object at 0x16a8a7200>
K_search_settings: {'lock': <unlocked _thread.lock object at 0x16a8a4b80>, 'k_search_method': 'linear', 'sill_thresh': 0.9, 'H_s

/Users/maksim/Desktop/Code/telf_internal/TELF/factorization/NMFk.py:856: UserWarning: calculate_error is True! Error calculation can make the runtime longer and take up more memory space!
  warnings.warn(


In [4]:
Ks = range(2,6,2)
name = "Example"
note = "This is an example run of NMFk"
results = model.fit(X, Ks, name, note)
print(results.keys())

  0%|          | 0/2 [00:00<?, ?it/s]/Users/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/maksim/miniconda3/envs/TELF/lib/python3.11/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 2/2 [00:00<00:00, 56.70it/s]


dict_keys(['time', 'k_predict', 'W', 'H', 'other_results'])
